In [ ]:
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split, GroupKFold, KFold, TimeSeriesSplit, StratifiedKFold
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from google.colab.patches import cv2_imshow
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dropout,Flatten,Dense,Input,Model
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from tensorflow.keras.callbacks import *
from tensorflow.keras import backend as K
from collections import deque
from imutils import paths
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2
import sys
import os
import gc
import matplotlib
matplotlib.use("Agg")

import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
class CyclicLR(Callback):
    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1 / (2. ** (x - 1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma ** (x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.

    def clr(self):
        cycle = np.floor(1 + self.clr_iterations / (2 * self.step_size))
        x = np.abs(self.clr_iterations / self.step_size - 2 * cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr - self.base_lr) * np.maximum(0, (1 - x)) * self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr - self.base_lr) * np.maximum(0, (1 - x)) * self.scale_fn(
                self.clr_iterations)

    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())

    def on_batch_end(self, epoch, logs=None):

        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

        K.set_value(self.model.optimizer.lr, self.clr())

In [ ]:
MIN_LR = 1e-6
MAX_LR = 1e-4
BATCH_SIZE = 16
STEP_SIZE = 8
CLR_METHOD = "triangular"
NUM_EPOCHS = 10
NUM_FOLDS = 5

INPUT_PATH = ""
OUTPUT_PATH = ""

In [ ]:
print("[INFO] Resimler Yükleniyor...")
imagePaths = list(paths.list_images(INPUT_PATH))
data = []
labels = []

for imagePath in tqdm(imagePaths):
  # extract the class label
    label = imagePath.split(os.path.sep)[-2]
    if label=="deprem":label=1
    else:label=0

    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))

    data.append(image)
    labels.append(label)

print("[INFO] Veri işleniyor...")
data = np.array(data, dtype="float32")
labels = np.array(labels)

In [ ]:
aug = ImageDataGenerator(
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

def get_model():
    baseModel = VGG16(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))

    headModel = baseModel.output
    headModel = Flatten(name="flatten")(headModel)
    headModel = Dense(512, activation="relu")(headModel)
    headModel = Dropout(0.5)(headModel)
    headModel = Dense(1, activation="sigmoid")(headModel)

    model = Model(inputs=baseModel.input, outputs=headModel)

    for layer in baseModel.layers:
        layer.trainable = False

    print("[INFO] Model eğitiliyor...")
    opt = SGD(lr=MIN_LR, momentum=0.9)
    model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])
    return model

### ONE FOLD ###
stepSize = STEP_SIZE * (data.shape[0] // BATCH_SIZE)
clr = CyclicLR(
    mode="triangular",
    base_lr=1e-6,
    max_lr=1e-4,
    step_size=stepSize)

model = get_model()
model.load_weights(OUTPUT_PATH+f"model.h5")

cp = ModelCheckpoint(OUTPUT_PATH+f"model.h5", monitor='loss', mode='min',save_best_only=True, verbose=1, period=1)
cp.set_model(model)

model.fit_generator(
    aug.flow(data, labels, batch_size=BATCH_SIZE),
    steps_per_epoch=data.shape[0] // BATCH_SIZE,
    epochs=NUM_EPOCHS,
    callbacks=[clr,cp],
    verbose=1)